# IE 360 Project Report

## Solar Power Forecasting

- Nergiz Selin Angın - 
- Halis Oğuz - 2018402129
- Mehmet Özer - 2016402129

## TABLE OF CONTENTS
* [Introduction](#introduction)
* [Descriptive Analysis](#descriptive_analysis)
* [Approach](#approach)
* [Results](#results)
* [Conclusion and Future Work](#conclusion)
* [Code](#code)

### INTRODUCTION

The problem is to predict the hourly production values of a solar energy plant located in Turkey. A regression approach combined with a time-series model was the first proposition to be used by the group. The data set consisted of hourly production values and weather conditions of the given hours. The data set was in long format, maditr package’s dcast function was used to convert the data to wide format.

Also, there were missing production values for 7 days, probably caused by some maintenance or database issue. Therefore, an average of 6 values, the same hour’s production values for the 3 days before and 3 days after is calculated for the corresponding hour and day’s production value. As an example, Şub 20 (February 20) is missing in the data, and its values is filled with mean of corresponding hourly values of past and following 3 days.

![missing_values](missing_values.png)

The brief explanations for the variables in the data set are provided below:
* TEMP: Temperature in units of Kelvin
* REL_HUMIDITY: Relative Humidity, in terms of percentage.
* DSWRF: Downward Shortwave Radiation Flux
* CLOUD_LOW_LAYER: Total cloud cover data for low-level type of clouds, in terms of percentage.


There remains one last problem with the target variable after the manipulations given above are applied. The facility was not operating with the full capacity at the time horizon of interest. It is likely that there were additional installations of solar panels/ or some maintenance done during some times that they would cause an unexplainable step drop or increase in the observed production values. A smoothing should be applied to get a picture of the true level at a specific time in the time horizon. A moving average method with parameter k=21 is used which added a column to the data to indicate the production capacity as a variable. 10 days after and before values with the center value is used with an average with equal weights so that one could obtain a clear and unbiased variable for the true production level at that time. The maximum of 21 days and k=11 days period is also evaluated for usage however k=21 average for center value yielded the best results for explaining the production level overall. Thus, k=21 center moving average smoothing is used. The first 10 and last 10 days have no variable due to the fact that they do not have 10 days ahead of them so the first obtained and last obtained value is duplicated. It has been assumed that they provide the least biased information regarding the capacity. 

The solar panel production does not necessarily display a monthly or quarterly seasonality but there was an hourly seasonality during the day. Although the seasons affect the production during a specific hour due to the change in the sunrise, sunset times and the angular degree of the sunlight rays hitting the panels. The data set was not providing a system with stable state but rather it was a system with transient states. Therefore, the evaluation was much harder due to the lack of data with stationarity. 

Forming 24 distinct models for each hour and evaluating their performances respectively is the approach decided to be executed to this problem. Also, an ARIMA model is formed to the residuals of the regression models formed for each hour to see if there are some auto-regressive components or moving average components that could be used to explain the residuals even further.

### Descriptive Analysis <a class="anchor" id="descriptive_analysis"></a>

Original time series data consist of hourly production values for dates from 2021-02-01 to two days before current date. Analyzing the data, we can see that there is an upper limit for the production values, of which production values cannot exceed. Construction of the production facility started in beginning of 2021. For this reason, capacity is small at the beginning of the data and increasing constantly until June 2021. Then, it is constant for some time around 40KWp. It decreases to 35KWp in the beginning of 2022 and stays on that level since (until recently). This maximum capacity change is a significant problem to deal with because linear regression works in an additive manner and it will find the optimal coefficients for an assumed maximum capacity, which will not work well with capacity change.

![production_values](production_values.png)

We are also given a weather data, which consists of previous weather values and future forecasts for some categories. These categories are Cloud Low Layer, Relative Humidity, Temperature and DSWRF, each are given their hourly values for 9 different locations. Values of a specific category for different locations are highly correlated because locations are close to each other. Therefore, using values for all locations is not necessary, and might lead to a possible overfitting.

![sample_data](sample_data.png)

Importance of values from each location might differ based on the hour of the day we are aiming to predict. For example, cloud low layer value at facility coordinate is important for hour 12. However, at hour 16, angle of the sunlight changes and sunlight reaching to the facility passes not from the clouds directly above the facility, but from the clouds around that area. Thus, cloud low layer value of another coordinate will be important for this hour.

Cloud low layer values and production values are negatively correlated. This is because if the sky is cloudy, sunbeams can’t pass through, therefore decreasing the production.

DSWRF values and production values are positively and significantly correlated. They will be useful to predict production values in our model.

Relative humidity values are negatively correlated with production. and temperature values are positively correlated with production.


![correlations](correlations.png)

Newly created smoothed production variable (named as 'capacity' in the code script) is highly correlated (=0.789) with the production value. This variable will be important for our prediction. Only problem is that this variable takes the mean of previous and future 21 days’ production values and fills the remaining values with the last day’s smoothed production value. If we want to predict values near a season change, this variable will create bias and result in an overprediction or underprediction.


![smoothed_production_correlation](correlation_smoothed_production.png)

There is no production at hours 0, 1, 2, 3, 4, 20, 21, 22 and 23. There is a negligible production at hour 5 only for few days. For these reasons, and because all the hours have their unique patterns (hourly seasonality) hourly models are created for hours 6 to 19.


### Approach <a class="anchor" id="approach"></a>

For hours from 6 to 19, a different model is established for each hour. Considering the historical data, the production of all the remaining hours is considered to be zero. The grid search method is used to determine the parameters to be used in each model. "Set.seed(500)" is used in each grid search to get the same values in each attempt. 4 different weather variables are provided in the data for each of the 9 coordinates. This will make 9 coordinates multiplied with 4 variables, 36 weather variables. When we try to set up a linear model with these weather variables, using more than 2 location values of the same variable disturbs the model. The coefficients of the same variable can appear in opposite signs for different locations. That's why the maximum 2 coordinate values of the same weather variable are evaluated when doing a grid search. Since a separate model was established for each hour, a separate grid search is performed for all the parameters to be used for each hour. 1000 different models using different weather variable combinations are compared according to weighted mean absolute percentage errors and the variables giving the minimum WMAPE are selected for use in the model. 

The temperature and production variables’ interaction were peculiar in the sense that increasing temperature values up to a certain threshold would increase production however after a certain threshold excessive heat would deteriorate the production. One could say that after a specific value, not known, the production was affected negatively. Therefore, to evaluate the non-linear relation the square of temperature variable is added as a column, and the highest R squared value is searched. The absolute value of the residuals that are above a certain threshold’s correlation with the temperature variable is analyzed and it is seen that they have a negative correlation. Which means that increasing temperature decreases the residuals or increases the predicted value but it deviates from the real value (the residual is far away from 0, or its mean). Also decreasing temperature decreases the prediction greatly so that the bias increases.

Multiplication of temperature squared with absolute value of a temperature value above its mean subtracted from the temperature value, i.e. |ti-285|*ti^2, is used to explain the deterioration of high temperature values on production. It best describes the non-linear relation mentioned above and explains the temperature variable the best. Then a grid search has been used to find the best parameter (for the variable 285) such that it would explain the output the best. 

After determining which weather parameters will be used and the parameter related to the temperature, the model is set for each hour with the specified parameters. Then, the ARIMA model is built on the residuals of the models. The ARIMA model parameters are also found again with grid search. The autoregressive parameter (p) value is simulated from 0 to 15; the differencing parameter (d) value is simulated from 0 to 2; the moving average parameter value (q) is simulated from 0 to 15. The AIC values of the resulting models are compared, and the parameters that give the smallest AIC value are selected as the ARIMA parameters. 

Finally, the parameters of the ARIMA model are also determined, and it is decided whether the ARIMA model should be added to the main model by looking at the WMAPEs of both models.

### Results <a class="anchor" id="results"></a>

For evaluating the models, multiple metrics have been used in order to detect and attack distinct aspects of the model. Also, the evaluation metrics have been collected for a period of 30 days while training the model again with the additional one day at each step. Therefore, one could say that the training set has been increased by one day after each evaluation,addition of one more hour to each model, and total bias, absolute bias as well as weighted mean absolute deviation have been used for interpretation of the models. For instance, total bias could be 0 which means the test period overall had a close mean value when compared to the predictions however absolute bias could be very high which shows the variance has been estimated inaccurately. Data is overall on the same level as the predictions but they are distinctively apart from fitting onto each other. Weighted mean absolute deviation is a better metric for evaluating the model performances since it is not affected by small values and is scale free. However, total bias and mean absolute bias could also be used to scale the deviation of different hours’ values which would indicate the work to be committed for the different models in order to obtain the most efficient results. As an example, if the model for hour 7 is returning an absolute bias of 250 in contrast if the model for hour 9 is returning an absolute bias of 25 then an improvement in model for hour 7 is going to decrease the overall wmape more. 

![alt text](error_metrics.png "Error Metrics")

From the metrics above, one could argue that the hour 10,12,13 and 16’s models are providing biased estimates. Also, hour 7’s model is problematic in the sense that it is capturing the overall level but it deviates from the mean gigantically. The same is true for hour 15 and 17’s models. The problem here is that we have evaluated the models based on the hourly period therefore scales are not removed. From the table above, one could see that WMAPE of hour 19 is 0.2446 whereas its total absolute bias is 5.69. The model for hour 19 has larger WMAPE than the model for hour 14 which has 152 Total Absolute Bias. Therefore, here scale is not removed and WMAPE provides some statistics even if it is not as explanatory as the other two metrics. Total Absolute and Total Bias provide one with meaningful ways that model could be improved through.

### Conclusion and Future Work <a class="anchor" id="conclusion"></a>

As a result of our work, we found that the most important variable for each model turned out to be the smoothed production. This is because recent production values are correlated to tomorrow's production values, and it is unlikely that tomorrow will have drastically different values from the near past. Other predictors were expected to be important, however they turned out to be not that important. This is probably because their effect is more complex and not easy to interpret.

There could be some other methods and techniques that could yield better or worse results, a trial and error process could be conducted to expose a better approach. There are many things that can be improved by using larger data and different predictors. In order to develop the model, the effects of variables on real-life electricity production can be investigated and variable values could be used in different combinations and with different formulas.


### Code <a class="anchor" id="code"></a>

Data preparation and model training / prediction codes can be found using the links provided below.

* [Data Preparation](https://github.com/BU-IE-360/spring22-sencer4898/blob/gh-pages/files/Project/codes/data_preparation.r)
* [Model Training / Prediction](https://github.com/BU-IE-360/spring22-sencer4898/blob/gh-pages/files/Project/codes/automated_prediction.R)